In [123]:
import sys
import os
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../utils_meinhold')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../utils_zonca')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('../utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../VtoT')
import realtime_gp as rt
import numpy as np
import datetime 
import h5py
import pandas as pd
from pointingtools import compute_parallactic_angle, altaz2ha 
from planets import getlocation
import warnings
from astropy.coordinates import AltAz, Angle, EarthLocation, ICRS, SkyCoord, frame_transform_graph
from astropy import units as u
import ephem
import matplotlib.pyplot as plt
import time
from datetime import datetime
import planets

import Tkinter,tkFileDialog

from Tkinter import *
import ttk

import pickle
import glob

import cPickle

%pylab


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\users\labuser\anaconda3\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Text', 'Button', 'datetime', 'Widget', 'var']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [ ]:
%matplotlib auto

In [2]:
def get_pointing_files(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    return filelist

In [3]:
def get_file_times(fld):
    startfile = fld[0][:43]+fld[0][49:-2]+'dat'
    endfile = fld[-1][:43]+fld[-1][49:-2]+'dat'
    
    #starttime = os.path.getctime(startfile)
    starttime= os.stat(startfile).st_mtime + 7*3600
    starttime = datetime.datetime.fromtimestamp(starttime)
    
    #endtime = os.path.getctime(endfile)
    endtime= os.stat(endfile).st_mtime + 7*3600
    endtime = datetime.datetime.fromtimestamp(endtime)
    
    return starttime, endtime

In [4]:
def fileStruct(n_array, chan, starttime, endtime):

    fpath = "C:/software_git_repos/polaris/polaris_data/level1"
    os.chdir(fpath)
    yrmoday = starttime.strftime('%Y%m%d')
    path = '-'.join((starttime.strftime('%H_%M_%S'), endtime.strftime('%H_%M_%S')+'222222222'))
    path = '-'.join((chan, path))
    print 'start time: ', starttime
    print 'end time: ', endtime
    print 'elapsed time: ', (endtime - starttime).total_seconds(), 'sec'
    if not os.path.exists(yrmoday):#this is the first file being created for that time
        os.makedirs(yrmoday)
        #set index to 0

    path = '/'.join((yrmoday,path))
    path = '.'.join((path,"h5"))
    with h5py.File(str(path).replace("pkl","h5"), mode="w") as f:
        f.create_dataset("data", data=n_array.to_records(index=False))

In [5]:
def round_fraction(number, res):
    amount = int(number/res)*res
    remainder = number - amount
    return amount if remainder < res/2. else amount+res

In [10]:
def read_some_data(datadict,datalabel,datatype='demod', filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        if datatype=='demod':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='D://software_git_repos/Polaris/polaris_data/demod_data/',parent=root,title='Choose a set of files'))
        if datatype=='raw':
            filelist = list(tkFileDialog.askopenfilenames(\
            initialdir='D://software_git_repos/Polaris/polaris_data/data/',parent=root,title='Choose a set of files'))            
        root.destroy()
    filelist.sort()
    
    if datatype=='demod':
        dlist=[]
        for f in filelist:
            hf=h5py.File(f)
            dlist.append(hf['demod_data'])
        d=concatenate(dlist)
        hf.close() 
        
    if datatype=='raw':
        d=rt.demod.read_raw(filelist)
    datadict[datalabel]=d
    return

In [99]:
def read_some_data2(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    dlist=[]
    for f in filelist:
        hf=h5py.File(f)
        dlist.append(hf['demod_data'])
    d=np.concatenate(dlist)
    hf.close() 

    datadict=d
        
    return datadict, filelist

In [31]:
def get_file_times(fld):
    startfile = fld[0][:43]+fld[0][49:-2]+'dat'
    endfile = fld[-1][:43]+fld[-1][49:-2]+'dat'
    
    #starttime = os.path.getctime(startfile)
    starttime= os.stat(startfile).st_mtime + 7*3600
    starttime = datetime.datetime.fromtimestamp(starttime)
    
    #endtime = os.path.getctime(endfile)
    endtime= os.stat(endfile).st_mtime + 7*3600
    endtime = datetime.datetime.fromtimestamp(endtime)
    
    return starttime, endtime

In [92]:
def plotnow_azrevsig(data, az, chan, var, res = 1.0, minmax=None,supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=get_h5_pointing(flp)
    #dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=get_all_demodulated_data(fld_demod, fld)
    #combined=combine_cofe_h5_pointing(dd,pp)

    #synchronized data and az values
    az1 = az
    data1 = data
    steps = len(data1)

    #convert to temp for cryo sensors
    if chan == 12:
        data1 = convert.convert(data1, 'i')
    if chan == 13:
        data1 = convert.convert(data1, 'e')
    if chan == 14:
        data1 = convert.convert(data1, 'h')
    if chan == 15:
        data1 = convert.convert(data1, 'l')

    #resolution
    dx = res
    dy = res

    #set up empty lists to append each revolution to
    data = []
    az = []
    iaz = [0]
    rev = 0

    #determine indices in azimuth/data array which correspond to a new revolution of the telescope
    for i in range(steps):
        #round values to resolution for comparison later
        az1[i] = round_fraction(az1[i], dx)
        if i > 0:
            if abs(az1[i] - az1[i-1]) >= 180.:
                iaz.append(i)
                rev += 1

    #append each revolution array to a list	    
    for j in range(rev):
        az.append(az1[iaz[j]:iaz[j+1]])
        data.append(data1[iaz[j]:iaz[j+1]])

    #append the last revolution
    data.append(data1[iaz[-1]:])
    az.append(az1[iaz[-1]:])
    rev += 1

    data = np.asarray(data)
    az = np.asarray(az)

    #create grid for plotting
    x, y = np.arange(0., 360.+dx, dx), np.arange(0., rev - 1 + dy, dy)
    AZ, REV = np.meshgrid(x, y)

    #set up empty array
    z = np.zeros(len(x)*len(y))
    sig = np.reshape(z, (len(y), len(x)))

    #small number for comparing floats
    epsilon = 1e-6

    #fill signal array with data points
    for r in range(rev):
        for a in range(len(x)):
            #find indices where combined azimuth data fits on x grid
            idx = np.where(abs(az[r] - x[a]) < epsilon)[0]
            #if idx length is 0 this will create a mask on that point, in idx len > 1, avg data points in the same bin
            sig[r][a] = data[r][idx].mean()

    #mask invalid values, i.e. where there are no data points
    sig = ma.masked_invalid(sig)

    #change units on plot label
    if int(chan[2:]) < 12:
        unit = 'V'
    else:
        unit = 'K' 

    name = rt.chantoname(chan)

    plt.pcolormesh(AZ, REV, sig)
    plt.colorbar(label = 'Signal, %s' % unit)

    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data1.min(),data1.max())
    plt.axis([0., 360., 0., rev - 1])
    plt.ylabel('revolution #')
    plt.xlabel('azimuth (deg)')
    plt.title('%s %s data binned to azimuth and revolution #' % (name, var))
    plt.grid()
    plt.show()

In [82]:
ddict={}

In [83]:
read_some_data(ddict,'sky_10-25',datatype='demod')

In [88]:
pp = {}

In [89]:
flp = get_pointing_files()
pp['sky_10-25'] = rt.get_h5_pointing(flp)

In [90]:
combined = rt.combine_cofe_h5_pointing(ddict['sky_10-25'], pp['sky_10-25'])

In [52]:
combined2 = rt.combine_cofe_h5_pointing(ddict['sky_11_18'], pp['sky_11_18'])

In [71]:
combined3 = rt.combine_cofe_h5_pointing(ddict['sky_10_30'], pp['sky_10_30'])

In [80]:
el = combined['el']
az = combined['az']
time = (combined['gpstime'] - combined['gpstime'][0])/1000.
chan = 'H1HiDC'
var = 'T'
data = (-combined['sci_data'][rt.nametochan(chan)][var]+4.021)
cooler = combined['sci_data'][rt.nametochan('Cooler')][var]*7 - 0.06
amplifier = combined['sci_data'][rt.nametochan('Amplifier')][var]*7 + 0.03

In [81]:
figure()
plot(time, data, lw=2, label='%s + 4.021 V' % chan)
#plot(time, cooler, lw=2, label='Cooler*7 - 0.06 V')
#plot(time, amplifier, lw=2, label='Amplifier*7 + 0.03 V')

legend()
xlabel('Seconds since start')
ylabel('Volts')
title('constant el scan starting from 11/13 03:20')

Text(0.5,1,'constant el scan starting from 11/13 03:20')

In [64]:
el = combined2['el']
az = combined2['az']
time = (combined2['gpstime'] - combined2['gpstime'][0])/1000.
chan = 'H1HiDC'
var = 'T'
data = (-combined2['sci_data'][rt.nametochan(chan)][var]+0.554)
cooler = (combined2['sci_data'][rt.nametochan('Cooler')][var] + 0.047)
amplifier = (combined2['sci_data'][rt.nametochan('Amplifier')][var])


In [65]:
figure()
plot(time, data, lw=2, label='%s + 0.554 V' % chan)
plot(time, cooler, lw=2, label='Cooler + 0.047 V')
plot(time, amplifier, lw=2, label='Amplifier')

legend()
xlabel('Seconds since start')
ylabel('Volts')
title('constant el scan starting from 11/18 20:27')

Text(0.5,1,'constant el scan starting from 11/18 20:27')

In [77]:
el = combined3['el']
az = combined3['az']
time = (combined3['gpstime'] - combined3['gpstime'][0])/1000.
chan = 'H1HiDC'
var = 'T'
data = (-combined3['sci_data'][rt.nametochan(chan)][var]+0.49)
cooler = (combined3['sci_data'][rt.nametochan('Cooler')][var])
amplifier = (combined3['sci_data'][rt.nametochan('Amplifier')][var])


In [79]:
figure()
plot(time, data, lw=2, label='%s + 0.49 V' % chan)
plot(time, cooler, lw=2, label='Cooler')
plot(time, amplifier, lw=2, label='Amplifier')

legend()
xlabel('Seconds since start')
ylabel('Volts')
title('constant el scan starting from 10-30 09:25')

Text(0.5,1,'constant el scan starting from 10-30 09:25')

In [101]:
ddict={}

In [103]:
dd, fld = read_some_data2()

In [104]:
flp = get_pointing_files()

In [105]:
pp = rt.get_h5_pointing(flp)

In [106]:
combined = rt.combine_cofe_h5_pointing(dd, pp)

In [107]:
starttime, endtime = rt.get_file_times(fld)
dt , ut= rt.convert_gpstime(starttime, combined['gpstime']/1000, format='datetime')

In [117]:
el = combined['el']
az = combined['az']
time = (combined['gpstime'] - combined['gpstime'][0])/1000.
chan = 'H1HiAC'
var = 'T'
data = (combined['sci_data'][rt.nametochan(chan)][var])

In [109]:
plotnow_azrevsig(data, az, rt.nametochan(chan), var, res=1.0, minmax = [data.min()/20., data.max()/20.])

c:\users\labuser\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:78: RuntimeWarning: Mean of empty slice.


In [112]:
icmin = 320000
icmax = 345000

In [127]:
figure()
plot(cdata)

In [121]:
imin = np.where(data==data[icmin:icmax].min())[0][0]
print imin
print data[imin]
print data[icmin:icmax].min()

332174
-0.1959526538848877
-0.1959526538848877


In [124]:
cra = 15*(5 + (34./60.) + (31.94/3600.))
cdec = 22 + (52.2/3600.)

caz, cel = planets.radec_to_azel(cra, cdec, 'UCSB', datetime.datetime.fromtimestamp(ut[imin]))

In [125]:
print cdec, cra
print cel, caz

22.0145 83.6330833333
-28.1597557577 28.0740863928


In [126]:
print az[imin], el[imin]

112.0 70.11089236625
